In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
from skimage.io import imread
from skimage.color import rgb2hsv
from skimage.feature import structure_tensor_eigvals, structure_tensor, hessian_matrix_eigvals,hessian_matrix,hog
from scipy.ndimage import gaussian_filter,gaussian_laplace,gaussian_gradient_magnitude
from skimage.filters.rank import entropy
import warnings
warnings.filterwarnings('ignore')

In [2]:
def Generate_Features(I):
    Features=[]
    filters=[]
    
    
    # RGB - Color Information
    filters.append(I[:,:,0])
    filters.append(I[:,:,1])
    filters.append(I[:,:,2])
    
    
    # HSV - Color Information

    img_hsv=rgb2hsv(I)
    H=img_hsv[:,:,0]
    S=img_hsv[:,:,1]
    V=img_hsv[:,:,2]
    
    filters.append(H)
    filters.append(S)
    filters.append(V)
    
    # ENTROPY
    
    N=[3,7,9,13,17,21]
    for n in N: 
        E=entropy(V,selem=np.ones([n,n]))
        E=E/np.max(E)
        filters.append(E)
    
    # GAUSSIAN
    
    sigmas = [0.7, 1, 1.6, 3.5, 5, 10]
    
    for sigma in sigmas:
        Gaussian = gaussian_filter(V, sigma=sigma)
        filters.append(Gaussian)
 

    # GAUSSIAN LAPLACE

    for sigma in sigmas:
        Gaus_Lapl = gaussian_laplace(V, sigma=sigma)
        filters.append(Gaus_Lapl)
        
    # GAUSSIAN GRADIENT MAGNITUDE

    for sigma in sigmas:
        GGM = gaussian_gradient_magnitude(V, sigma=sigma)
        filters.append(GGM)

    # STRUCTURE TENSOR

    for sigma in sigmas:
        st_1,st_2= structure_tensor_eigvals(*structure_tensor(V, sigma))
        filters.append(st_1)
        filters.append(st_2)      

    # HESSIAN MATRIX

    for sigma in sigmas:
        h_1,h_2 = hessian_matrix_eigvals(*hessian_matrix(V, sigma, order="rc"))
        filters.append(h_1)
        filters.append(h_2)     

        
    Features=np.array(filters)
    return Features

In [3]:
def Quality_Evaluation(Result,Ground_Truth):

    #Calculate Precision, Completeness-Recal and Quality metrics
    #for Quality Evaluation of our algorithm
    
    TP=np.sum(Result & Ground_Truth)
    FP=np.sum(Result & ~Ground_Truth)
    FN=np.sum(~Result & Ground_Truth)
    
    Precision= TP / ( TP + FP )
    Completeness= TP / ( TP + FN )
    Quality= TP / ( TP + FP + FN )
    return Precision, Completeness, Quality